In [1]:
import sys, re, math, time
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
from os import path
import os,binascii
import collections
from collections import OrderedDict
from matplotlib.pyplot import cm
%load_ext autoreload
%autoreload 2
from run_experiments import *
from datahelper import *
import keras.metrics
keras.metrics.cindex_score = cindex_score
from sklearn.metrics import mean_squared_error, f1_score
from keras.models import load_model
from sklearn.model_selection import ParameterGrid
%matplotlib inline

Using TensorFlow backend.


In [4]:
FLAGS = argparser()
FLAGS.log_dir = FLAGS.log_dir + str(time.time()) + "/"
FLAGS.num_windows = [32]
FLAGS.seq_window_lengths = [4, 6]
FLAGS.smi_window_lengths = [4]
FLAGS.batch_size = 256
FLAGS.num_epoch = 100
FLAGS.max_seq_len = 1000
FLAGS.max_smi_len = 95
FLAGS.dataset_path= '../data/davis/'
FLAGS.problem_type = 1
FLAGS.bns = [True]
FLAGS.dropouts = [0.1, 0.2, 0.3]
FLAGS.charseqset_size = CHARPROTLEN 
FLAGS.charsmiset_size = CHARISOSMILEN 
dtc_data_file = '../../data/dtc_for_deepDTA.csv'

In [5]:
proteins = json.load(open(FLAGS.dataset_path+"proteins.txt"), object_pairs_hook=OrderedDict)
proteins = [i[1] for i in proteins.items()]
dtc_train = pd.read_csv(dtc_data_file)
with open('proteins.csv', 'w') as handler:
    for i, seq in enumerate(list(dtc_train['fasta'].unique()) + proteins):
        handler.write('>id'+str(i)+'\n')
        handler.write(seq+'\n')

In [7]:

dataset = DataSet( fpath = FLAGS.dataset_path,
                      setting_no = FLAGS.problem_type,
                      seqlen = FLAGS.max_seq_len,
                      smilen = FLAGS.max_smi_len,
                      need_shuffle = False )
# set character set size

XD, XT, Y = dataset.parse_data(fpath = FLAGS.dataset_path)
XD, XT, Y = np.asarray(XD), np.asarray(XT), np.asarray(Y)

label_row_inds, label_col_inds = np.where(np.isnan(Y)==False)  #basically finds the point address of affinity [x,y]

if not os.path.exists(figdir):
    os.makedirs(figdir)

Y = np.mat(np.copy(Y))
train_drugs, train_prots,  train_Y = prepare_interaction_pairs(XD, XT, Y, label_row_inds, label_col_inds)


XD_dtc, XT_dtc, Y_dtc = get_DTC_train(dtc_data_file, FLAGS.max_smi_len, FLAGS.max_seq_len)

all_train_drugs = np.concatenate((np.asarray(train_drugs), np.asarray(XD_dtc)), axis=0)
all_train_prots = np.concatenate((np.asarray(train_prots), np.asarray(XT_dtc)), axis=0)
all_train_Y = np.concatenate((np.asarray(train_Y), np.asarray(Y_dtc)), axis=0)
all_train_Y = -np.log10(1e-8+all_train_Y/1e9)

val_inds = get_n_fold_by_drugs(all_train_drugs, n_splits=10)

Reading ../data/davis/ start
Read ../data/davis/ start


In [9]:
n_repeats = 1
param_grid = ParameterGrid([
    {
        'num_filters': FLAGS.num_windows,
        'smi_filter_length': FLAGS.smi_window_lengths,
        'seq_filter_length': FLAGS.seq_window_lengths,
        'dropout': FLAGS.dropouts,
        'apply_bn': FLAGS.bns
    }
])

if path.exists(FLAGS.results_pickle):
    with open(FLAGS.results_pickle, 'rb') as handle:
        results = pickle.load(handle)
else:
    results = {}

for ind in range(len(param_grid)):
    
    params = param_grid[ind]
    
    param_name = str(binascii.b2a_hex(os.urandom(15))).replace("'", '')

    early_stopping_callback = CustomStopper(monitor='val_loss', patience=10, start_epoch=20)
    fold_id, avg_loss, avg_cindex, avg_f1, avg_rmse = 0, 0.0, 0.0, 0.0, 0.0
    for tr_fold, val_fold in val_inds.split():
        print(fold_id)
        model_name='checkpoints/davis_dtc_dta_'+param_name+'fold'+str(fold_id)

        checkpoint_callback = ModelCheckpoint(model_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

        if fold_id >= n_repeats:
            break
        XD_train, XT_train, Y_train = all_train_drugs[tr_fold], all_train_prots[tr_fold], all_train_Y[tr_fold]
        XD_val, XT_val, Y_val = all_train_drugs[val_fold], all_train_prots[val_fold], all_train_Y[val_fold]

        gridmodel = build_combined_categorical(FLAGS, params['num_filters'], params['smi_filter_length'], 
                                       params['seq_filter_length'], params['dropout'], apply_bn=params['apply_bn'])

        gridres = gridmodel.fit(([XD_train, XT_train ]), Y_train, batch_size=FLAGS.batch_size, epochs=FLAGS.num_epoch, 
                validation_data=( ([np.array(XD_val), np.array(XT_val) ]), np.array(Y_val))
                   ,callbacks=[early_stopping_callback, checkpoint_callback], verbose=2)
        
        gridmodel = load_model(model_name+'.h5')
        predicted_labels = gridmodel.predict([np.array(XD_val), np.array(XT_val)])[:, 0]
        loss, rperf2 = gridmodel.evaluate(([np.array(XD_val),np.array(XT_val) ]), np.array(Y_val), verbose=0)
        avg_loss += loss/n_repeats
        avg_cindex += rperf2/n_repeats
        avg_f1 += f1_score(Y_val>7, predicted_labels>7)/n_repeats
        avg_rmse += np.sqrt(mean_squared_error(Y_val, predicted_labels))/n_repeats
        fold_id += 1
    
    results[param_name] = {'FLAGS':FLAGS, 'params': params, 'loss': avg_loss, 'cindex': avg_cindex,
                       'rmse':  avg_rmse, 'f1': avg_f1, 'n_repeats': n_repeats}
    with open(FLAGS.results_pickle, 'wb') as handle:
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

0
Train on 44175 samples, validate on 4292 samples
Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[256,1000,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node embedding_4/GatherV2 (defined at /home/ibrahim/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:1188)  = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@training_1/Adam/gradients/embedding_4/GatherV2_grad/Reshape"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding_4/embeddings/read, _arg_input_4_0_4/_445, embedding_4/GatherV2/axis)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_1/mul/_509}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2319_loss_1/mul", tensor_type=DT_FLOAT, _devi

In [ ]:
XD_test, XT_test = get_DTC_train(data_path+'dtc_test_for_deepDTA.csv', FLAGS.max_smi_len, FLAGS.max_seq_len, with_label=False)
XD_test, XT_test = np.asarray(XD_test), np.asarray(XT_test)
predicted_labels = gridmodel.predict([np.array(XD_test), np.array(XT_test) ])


In [ ]:
submission_data = pd.read_csv(data_path+'round_1_template.csv')
submission_data.loc[:, 'pKd_[M]_pred'] = predicted_labels
submission_data.to_csv(data_path+'submission_file.csv', index=None)